<a href="https://colab.research.google.com/github/umersajid11/RAGAJKTourisam/blob/main/hackathonepecversion_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio langchain sentence-transformers groq huggingface-hub pyaudio numpy faiss-cpu PyPDF2 transformers

In [ ]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    pdf_reader = PyPDF2.PdfReader(pdf_path)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

# Replace with the path to your dataset
pdf_text = extract_text_from_pdf("/content/Dataset.pdf")



In [ ]:
from sentence_transformers import SentenceTransformer

# Load pre-trained embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Chunk text into smaller sections
def chunk_text(text, chunk_size=512):
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

chunks = chunk_text(pdf_text)
chunk_embeddings = embedding_model.encode(chunks)


In [ ]:
import faiss
import numpy as np

# Create FAISS index
dimension = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(chunk_embeddings))


In [ ]:
import os
from groq import Groq

os.environ["GROQ_API_KEY"] = "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"
client = Groq(api_key=os.environ["GROQ_API_KEY"])

def query_llama(prompt):
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama3-70b-8192",
        stream=False,
    )
    return response.choices[0].message.content


In [ ]:
def retrieve_context(query):
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), 1)
    return chunks[indices[0][0]]

def chatbot_response(query):
    context = retrieve_context(query)
    prompt = f"Context: {context}\nQuestion: {query}\nAnswer:"
    return query_llama(prompt)


In [ ]:
import gradio as gr

chat_history = []

def chat_interface(user_query):
    global chat_history
    bot_response = chatbot_response(user_query)
    chat_history.append((user_query, bot_response))
    return chat_history

with gr.Blocks() as demo:
    with gr.Row():
        gr.Markdown("### RAG Chatbot for AJK Tourism")
    with gr.Column():
        chatbot = gr.Chatbot()
        query = gr.Textbox(label="Your Query")
        submit_button = gr.Button("Submit")

    submit_button.click(chat_interface, inputs=query, outputs=chatbot)

demo.launch()


/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:242: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d8e1c17c0ab604948a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
